In [ ]:
import sys
sys.path.append("../..")

import os
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import time

import skopi as sk
import skopi.gpu as sg

In [ ]:
# Create a particle object
particle = sk.Particle()

particle.read_pdb('../input/3iyf.pdb', ff='WK')

# Load beam
beam = sk.Beam('../input/exp_chuck.beam') 

# Load and initialize the detector
det = sk.PnccdDetector(geom = '../lcls/amo86615/PNCCD::CalibV1/Camp.0:pnCCD.1/geometry/0-end.data', 
                       beam = beam)

# Calculate the 3D diffraction volume

In [ ]:
mesh_length = 151

# Setup the reciprocal space mesh grid for the detector
# The edge pixel resolution is slightly larger than the corner resolution of the detector panel.
mesh, voxel_length= det.get_reciprocal_mesh(voxel_number_1d = mesh_length)

# Uniformly Take 10 slices

In [ ]:
file_name = 'imStack-temp.hdf5'
n = 100
if True:
    print "Restarting"
    
    # Generate n images uniformly over 4-sphere
    # Calculate the 3D diffraction intensity volume
    volume = sg.calculate_diffraction_pattern_gpu(mesh, particle, return_type='intensity')

    # Because the Intensity is very low, I would like to increase the intensity a little bit
    volume *= 10
    
    orientations = sk.geometry.get_uniform_quat(num_pts=n)
    slices = sk.geometry.take_n_slice(
        pattern_shape = det.pedestal.shape,
        pixel_momentum = det.pixel_position_reciprocal,
        volume = volume,
        voxel_length = voxel_length,
        orientations = orientations)
    
    vshape = volume.shape
    ishape = (n, 4, 512, 512)
    
    with h5.File(file_name, 'w') as f:
        f.create_dataset('volume', shape=vshape, maxshape=vshape, data=volume, dtype=np.float64)
        f.create_dataset('imUniform', shape=ishape, maxshape=ishape, data=slices, dtype=np.int32)
        f.create_dataset('imOrientations', orientations.shape, data=orientations, dtype=np.float32)

with h5.File(file_name, 'r') as f:
    print "Loading"
    
    volume = f['volume'][:]
    data2 = f['imUniform'][:]
    orientations = f['imOrientations'][:]

In [ ]:
plt.imshow(np.log(volume[:,mesh_length/2,:]))
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
for i in range(4):
    plt.subplot(1, 4, i+1)
    plt.imshow(np.log(data2[0, i]))
plt.show()

## Show quantizated results

In [ ]:
data2cq = det.add_correction_and_quantization_batch(pattern_batch=data2)

In [ ]:
plt.figure(figsize=(20, 8))
for i in range(4):
    plt.subplot(1, 4, i+1)
    plt.imshow(data2cq[0, i])
plt.show()

In [ ]:
data2a = det.assemble_image_stack_batch(image_stack_batch=data2cq)

In [ ]:
plt.imshow(data2a[0])
plt.show()

In [ ]:
l = 0
pixel_momentum = det.pixel_position_reciprocal

rot_mat = sk.geometry.quaternion2rot3d(orientations[l, :])

# rotate the pixels in the reciprocal space.
# Notice that at this time, the pixel position is in 3D
rotated_pixel_position = sk.geometry.rotate_pixels_in_reciprocal_space(rot_mat, pixel_momentum)
# calculate the index and weight in 3D
local_index, local_weight = sk.geometry.get_weight_and_index(
    pixel_position=rotated_pixel_position,
    voxel_length=voxel_length,
    voxel_num_1d=volume.shape[0])

In [ ]:
pattern_shape = det.pedestal.shape
pixel_num = np.prod(pattern_shape)

volume_num_1d = volume.shape[0]
convertion_factor = np.array([volume_num_1d * volume_num_1d, volume_num_1d, 1], dtype=np.int64)

index_2d = np.reshape(local_index, [pixel_num, 8, 3])
index_2d = np.matmul(index_2d, convertion_factor)

volume_1d = np.reshape(volume, volume_num_1d ** 3)
weight_2d = np.reshape(local_weight, [pixel_num, 8])

# Expand the data to merge
data_to_merge = volume_1d[index_2d]

# Merge the data
data_merged = np.sum(np.multiply(weight_2d, data_to_merge), axis=-1)

slice_ = np.reshape(data_merged, pattern_shape)

In [ ]:
plt.figure(figsize=(20, 4))
for i in range(4):
    plt.subplot(1, 4, i+1)
    plt.imshow(np.log(data2[0, i]))
plt.show()

In [ ]:
plt.figure(figsize=(20, 4))
for i in range(4):
    plt.subplot(1, 4, i+1)
    plt.imshow(np.log(slice_.astype('int32')[i]))
plt.show()

In [ ]:
plt.figure(figsize=(20, 4))
for i in range(4):
    plt.subplot(1, 4, i+1)
    plt.imshow(data2[0, i] - slice_.astype('int32')[i])
    plt.colorbar()
plt.show()

In [ ]:
data2cq = det.add_correction_and_quantization_batch(pattern_batch=data2)
data2a = det.assemble_image_stack_batch(image_stack_batch=data2cq)

## Merge intensities

In [ ]:
volume_merge = np.zeros_like(volume)
volume_weight = np.zeros_like(volume)

In [ ]:
pixel_momentum = det.pixel_position_reciprocal
volume_m_1d = np.reshape(volume_merge, volume_num_1d ** 3)
volume_w_1d = np.reshape(volume_weight, volume_num_1d ** 3)
    
for l in range(n):
    rot_mat = sk.geometry.quaternion2rot3d(orientations[l, :])

    # rotate the pixels in the reciprocal space.
    # Notice that at this time, the pixel position is in 3D
    rotated_pixel_position = sk.geometry.rotate_pixels_in_reciprocal_space(rot_mat, pixel_momentum)
    # calculate the index and weight in 3D
    local_index, local_weight = sk.geometry.get_weight_and_index(
        pixel_position=rotated_pixel_position,
        voxel_length=voxel_length,
        voxel_num_1d=volume.shape[0])
    
    index_2d = np.reshape(local_index, [pixel_num, 8, 3])
    index_2d = np.matmul(index_2d, convertion_factor)

    weight_2d = np.reshape(local_weight, [pixel_num, 8])
    
    data_1d = np.reshape(data2[l], pixel_num)

    # Expand the data to merge
    volume_m_1d[index_2d] += np.multiply(weight_2d, data_1d[:,np.newaxis])
    volume_w_1d[index_2d] += weight_2d

volume_merge = np.reshape(volume_m_1d, volume.shape)
volume_weight = np.reshape(volume_w_1d, volume.shape)

In [ ]:
plt.imshow(np.log(volume_weight[:,(mesh_length+1)//2]))
plt.show()

In [ ]:
plt.imshow(np.log(volume_merge[:,(mesh_length+1)//2]))
plt.show()

In [ ]:
volume_merge /= volume_weight

In [ ]:
plt.imshow(np.log(volume_merge[:,(mesh_length+1)//2]))
plt.show()

In [ ]:
plt.imshow(np.log(volume[:,(mesh_length+1)//2]))
plt.show()

In [ ]:
vshape = volume_merge.shape
with h5.File(file_name, 'a') as f:
    f.create_dataset('merged', shape=vshape, maxshape=vshape, data=volume_merge, dtype=np.float32)

## Merge photons

In [ ]:
volume_merge_ph = np.zeros_like(volume)
volume_weight_ph = np.zeros_like(volume)

In [ ]:
# Same with photons instead of intensities
pixel_momentum = det.pixel_position_reciprocal
volume_m_1d = np.reshape(volume_merge_ph, volume_num_1d ** 3)
volume_w_1d = np.reshape(volume_weight_ph, volume_num_1d ** 3)
    
for l in range(n):
    rot_mat = sk.geometry.quaternion2rot3d(orientations[l, :])

    # rotate the pixels in the reciprocal space.
    # Notice that at this time, the pixel position is in 3D
    rotated_pixel_position = sk.geometry.rotate_pixels_in_reciprocal_space(rot_mat, pixel_momentum)
    # calculate the index and weight in 3D
    local_index, local_weight = sk.geometry.get_weight_and_index(
        pixel_position=rotated_pixel_position,
        voxel_length=voxel_length,
        voxel_num_1d=volume.shape[0])
    
    index_2d = np.reshape(local_index, [pixel_num, 8, 3])
    index_2d = np.matmul(index_2d, convertion_factor)

    weight_2d = np.reshape(local_weight, [pixel_num, 8])
    
    data_1d = np.reshape(data2cq[l], pixel_num)

    # Expand the data to merge
    volume_m_1d[index_2d] += np.multiply(weight_2d, data_1d[:,np.newaxis])
    volume_w_1d[index_2d] += weight_2d

volume_merge_ph = np.reshape(volume_m_1d, volume.shape)
volume_weight_ph = np.reshape(volume_w_1d, volume.shape)

In [ ]:
volume_merge_ph /= volume_weight_ph

In [ ]:
plt.imshow(np.log(volume_merge_ph[:,(mesh_length+1)//2]))
plt.show()

In [ ]:
plt.imshow(np.log(volume[:,(mesh_length+1)//2]))
plt.show()

In [ ]:
vshape = volume_merge.shape
with h5.File(file_name, 'a') as f:
    f.create_dataset('photons', shape=vshape, maxshape=vshape, data=volume_merge_ph, dtype=np.float32)
    #f['photons'][:] = volume_merge_ph